In [1]:
import numpy as np
from src.utils import *
from tqdm import tqdm

operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp]
one_arg_op=[np.sin, np.cos, np.exp]

In [14]:
problem_0 = np.load("data/problem_0.npz")
x = problem_0["x"]
y = problem_0["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)

x.shape: (2, 1000)
y.shape: (1000,)


In [15]:
NUM_POPULATION = 60
my_population = create_population(NUM_POPULATION,5,x.shape[0])

In [16]:
cost_pop = cost_population(my_population,x,y)
print(len(my_population))
print("Cost population: ", cost_pop.mean())
print("Best cost: ", cost_pop.min())

59
Cost population:  6.678978862885219e+21
Best cost:  1.8910558431660756


In [17]:
min_index = np.argmin(cost_pop)
print("Best individual: ", my_population[min_index])

Best individual:  sin(x[0])


In [26]:
NUM_GENERATIONS = 21
DEDUPE_INTERVAL = 2
operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp]
one_arg_op=[np.sin, np.cos, np.exp]
XOVER = 0.5
MUTATION = 0.5

for generation in tqdm(range(NUM_GENERATIONS)):
    cost_pop = cost_population(my_population,x,y)
    best_ind, worst_ind = tournament_selection(cost_pop,15)
    
    while len(my_population) - len(worst_ind) <= NUM_POPULATION:
        if random.random() < XOVER:
            # Xover the best individual and replace worst individuals
            parent1 = my_population[best_ind[np.random.choice(len(best_ind))]]
            parent2 = my_population[best_ind[np.random.choice(len(best_ind))]]
            child1, child2 = crossover(parent1,parent2)
            my_population.append(child1)
            my_population.append(child2)

        if random.random() < MUTATION:
            # Mutate best individuals and replace worst individuals
            parent = my_population[best_ind[np.random.choice(len(best_ind))]]
            child = mutation(parent, x.shape[0])
            my_population.append(child)
    for ind in sorted(worst_ind, reverse=True):
        del my_population[ind]
        
    # Deduplicate every few iterations
    if generation % DEDUPE_INTERVAL == 0:
        print("Deduplicating population")
        my_population = list(dict.fromkeys(my_population))

100%|██████████| 21/21 [00:03<00:00,  5.71it/s]


In [24]:
print("my_population.shape:", len(my_population))
cost_pop = cost_population(my_population,x,y)
print("Cost population: ", cost_pop.mean())
print("Best cost: ", cost_pop.min())

my_population.shape: 61
Cost population:  0.5906985414885839
Best cost:  0.01067566400550203


In [25]:
min_index = np.argmin(cost_pop)
print("Best individual: ", my_population[min_index])

Best individual:  x[0]


In [21]:
# print the top 10 individuals witg lowest cost
sorted_index = np.argsort(cost_pop)
for i in range(30):
    print("Individual: ", my_population[sorted_index[i]])
    print("Cost: ", cost_pop[sorted_index[i]])
    print("")

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.01067566400550203

Individual:  x[0]
Cost:  0.0106756

In [22]:
a = [1,1,1,2,3,4,5,5,6,7,8]
a = list(dict.fromkeys(a))
print(a)

[1, 2, 3, 4, 5, 6, 7, 8]
